<h1 style="color:black; text-align:center; background-color:powderblue; padding:20px; margin:0"> <b>期末專題 : 人臉濾鏡(來源為github 網頁)</b></h1>
<h2 style="text-align:center; background-color:powderblue; padding:7px; margin:0"> Resource : Akshay Chandra Lagandula, IIT Hyderabad </h2>
<h3 style="text-align:center; background-color:powderblue; padding:7px; margin:0"> github webpage : <a href=https://github.com/akshaychandra21/Selfie_Filters_OpenCV>https://github.com/akshaychandra21/Selfie_Filters_OpenCV</a></h3>
<h4 style="text-align:center; background-color:powderblue; padding:7px; margin:0">Editor : B05203008 廖哲宏</h4>

<p style="color:blue; font-size:120%"><b>這是我的期末專題，由於我是一個人完成的，人力和時間上都比較吃緊，因此我決定改良一個簡單的人臉濾鏡。<br>我的程式架構來源如上所示，其原型僅能加上眼鏡，改良後可有不同的五官組合。</b></p>

### ● 程式原理：
1. 利用openCV結合網路攝影機，擷取串流影像並進行臉部偵測。
2. 將偵測到的臉resize至96x96之tensor，而後作為15標記點之五官偵測的input。
3. 利用事先 train 好的模型，predict出15個五官標記點。
4. 藉由標記點進行圖層(眼睛、鼻子、嘴巴)的更換。
5. 將編輯後的串流影像輸出。

<h2 style="background-color:pink; padding:20px;margin:0">1. Training model 的部分(此部分僅修改過method命名)</h2>
<h2 style="background-color:pink; padding:20px;margin:0">(我在remote server上跑，因此這裡沒有執行紀錄)</h2>
<h3 style="background-color:pink; padding:20px;margin:0">檔案原名:model_builder.py</h3>

In [ ]:
from utils import load_data
from my_CNN_model import *
import cv2


# Load training set
X_train, y_train = load_data()

# Set model layout
my_model = create_CNN_model()

# Compilt model
compile_CNN_model(my_model, optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])

# Training model
hist = train_CNN_model(my_model, X_train, y_train)

# Saving model
save_CNN_model(my_model, 'my_model')

<h2 style="background-color:LightSalmon; padding:20px;margin:0">2.Model layout(經修改後準確率從0.74.. 提升到0.82..)</h2>
<h3 style="background-color:LightSalmon; padding:20px;margin:0">檔案原名:my_CNN_model.py</h3>

In [7]:
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Convolution2D, MaxPooling2D, Dropout
from keras.layers import Flatten, Dense
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
import os

os.environ['CUDA_VISIBLE_DEVICES']='1'                                           ####設定目標 GPU####

def get_my_CNN_model_architecture():
    '''
    The network should accept a 96x96 grayscale image as input, and it should output a vector with 30 entries,
    corresponding to the predicted (horizontal and vertical) locations of 15 facial keypoints.
    '''
    model = Sequential()
    model.add(Convolution2D(32, (5, 5), input_shape=(96,96,1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(Dropout(0.2))

    model.add(Convolution2D(32, (2, 2), activation='relu'))
    model.add(Dropout(0.3))

    model.add(Flatten())

    model.add(Dense(64, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(30))

    return model;

def compile_my_CNN_model(model, optimizer, loss, metrics):
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

def train_my_CNN_model(model, X_train, y_train):
    return model.fit(X_train, y_train, epochs=100, batch_size=200, verbose=1, validation_split=0.2)

def save_my_CNN_model(model, fileName):
    model.save(fileName + '.h5')

def load_my_CNN_model(fileName):
    return load_model(fileName + '.h5')


<h2 style="background-color:Antiquewhite; padding:20px;margin:0">3.訓練資料提取與前處理(此處並未修改)</h2>
<h3 style="background-color:Antiquewhite; padding:20px;margin:0">檔案原名:utils.py</h3>

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from keras.models import load_model
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle

def load_data(test=False):
    """
    Loads data from FTEST if *test* is True, otherwise from FTRAIN.
    Important that the files are in a `data` directory
    """
    FTRAIN = 'data/training.csv'
    FTEST = 'data/test.csv'
    fname = FTEST if test else FTRAIN
    df = read_csv(os.path.expanduser(fname))  # load dataframes

    # The Image column has pixel values separated by space; convert
    # the values to numpy arrays:
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))

    df = df.dropna()  # drop all rows that have missing values in them

    X = np.vstack(df['Image'].values) / 255.  # scale pixel values to [0, 1] (Normalizing)
    X = X.astype(np.float32)
    X = X.reshape(-1, 96, 96, 1) # return each images as 96 x 96 x 1

    if not test:  # only FTRAIN has target columns
        y = df[df.columns[:-1]].values
        y = (y - 48) / 48  # scale target coordinates to [-1, 1] (Normalizing)
        X, y = shuffle(X, y, random_state=42)  # shuffle train data
        y = y.astype(np.float32)
    else:
        y = None

    return X, y


<h2 style="background-color:PaleGoldenRod; padding:20px;margin:0">4. 濾鏡執行程式(大部分修改於此處，將原本只有墨鏡之濾鏡增加到五官)</h2>
<h3 style="background-color:PaleGoldenRod; padding:20px;margin:0">檔案原名:shader.py</h3>

In [1]:
from my_CNN_model import *
import cv2
import numpy as np
import os
import math
import random

# Load the model built in the previous step
# 載入預先訓練好的模型
my_model = load_CNN_model('./folder2/8481acc-8224valacc')
print ("load model complete")

# Face cascade to detect faces
# 由 github提供之臉部偵測 xml檔
face_cascade = cv2.CascadeClassifier('cascades/haarcascade_frontalface_default.xml')
print ("load cascate complete")

# Define the upper and lower boundaries for a color to be considered "Blue"
# 定義碰到按鈕的藍色判斷範圍
blueLower = np.array([100, 60, 60])
blueUpper = np.array([140, 255, 255])

# Define a 5x5 kernel for erosion and dilation
# 用一個5x5convolution window
kernel = np.ones((5, 5), np.uint8)

# Define filters'images/mask1.png', 
imgList = os.listdir("./images")                       #將原本單一filter改為三種filter， -------------------------------#
sunglassFilter = []                                     #每次觸碰 Next filter按鈕時隨機選取三種 filter                   # 
noseFilter = []                                                                                                        # 
mouthFilter = []                                                                                                       # 
for file in imgList:                                                                                                   #
    if (file.startswith("sunglasses")):                                                                                #
        sunglassFilter.append(os.path.join("./images/", file))                                                         #
    elif (file.startswith("nose")):                                                                                    #
        noseFilter.append(os.path.join("./images/", file))                                                             #
    elif (file.startswith("mouth")):                                                                                   #
        mouthFilter.append(os.path.join("./images/", file))                                                            #
filtIdx1 = 0                                                                                                           # 
filtIdx2 = 0                                                                                                           # 
filtIdx3 = 0                                 ########################################################################### 

# Load the video
camera = cv2.VideoCapture(0)

# Keep looping
while (True):
    # Grab the current paintWindow
    (grabbed, frame) = camera.read()
    frame = cv2.flip(frame, 1)
    frame2 = np.copy(frame)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Add the 'Next Filter' button to the frame
    rec_x = 500
    rec_y = 400
    rec_w = 120
    rec_h = 55
    frame = cv2.rectangle(frame, (rec_x, rec_y), (rec_x + rec_w, rec_y + rec_h), (235,50,50), -1)
    cv2.putText(frame, "NEXT FILTER", (rec_x+12, rec_y+27), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.25, 6)

    # Determine which pixels fall within the blue boundaries and then blur the binary image
    blueMask = cv2.inRange(hsv, blueLower, blueUpper)
    blueMask = cv2.erode(blueMask, kernel, iterations=2)
    blueMask = cv2.morphologyEx(blueMask, cv2.MORPH_OPEN, kernel)
    blueMask = cv2.dilate(blueMask, kernel, iterations=1)

    # Find contours (bottle cap in my case) in the image
    (cnts, _) = cv2.findContours(blueMask.copy(), cv2.RETR_EXTERNAL,
    	cv2.CHAIN_APPROX_SIMPLE)
    center = None

    # Check to see if any contours were found
    if len(cnts) > 0:
    	# Sort the contours and find the largest one -- we
    	# will assume this contour correspondes to the area of the bottle cap
        cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
        # Get the radius of the enclosing circle around the found contour
        ((x, y), radius) = cv2.minEnclosingCircle(cnt)
        # Draw the circle around the contour
        cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
        # Get the moments to calculate the center of the contour (in this case Circle)
        M = cv2.moments(cnt)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))

        if rec_y <=center[1] <= (rec_y + rec_h):                             #每次觸碰按鈕，隨機生成三種 filter---------#
            if rec_x <= center[0] <= (rec_x+rec_w): # Next Filter                                                     #
                filtIdx1 = random.randint(0, len(sunglassFilter)-1)                                                   #
                filtIdx2 = random.randint(0, len(noseFilter)-1)                                                       #
                filtIdx3 = random.randint(0, len(mouthFilter)-1)                                                      #
                continue                                                    ###########################################

    for (x, y, w, h) in faces:

        # Grab the face
        gray_face = gray[y:y+h, x:x+w]
        color_face = frame[y:y+h, x:x+w]
        # Normalize to match the input format of the model - Range of pixel to [0, 1]
        gray_normalized = gray_face / 255

        # Resize it to 96x96 to match the input format of the model
        original_shape = gray_face.shape # A Copy for future reference
        face_resized = cv2.resize(gray_normalized, (96, 96), interpolation = cv2.INTER_AREA)
        face_resized_copy = face_resized.copy()
        face_resized = face_resized.reshape(1, 96, 96, 1)

        # Predicting the keypoints using the model
        keypoints = my_model.predict(face_resized)
        
        
        # De-Normalize the keypoints values
        keypoints = keypoints * 48 + 48

        # Map the Keypoints back to the original image
        face_resized_color = cv2.resize(color_face, (96, 96), interpolation = cv2.INTER_AREA)
        face_resized_color2 = np.copy(face_resized_color)

        # Pair them together
        points = []
        for i, co in enumerate(keypoints[0][0::2]):
            points.append((co, keypoints[0][1::2][i]))
        #print ("points shape: {}".format(points.shape))

        # Add FILTER to the frame                                                #分別將三種 filter讀入-------------------#
        filt1 = cv2.imread(sunglassFilter[filtIdx1], cv2.IMREAD_UNCHANGED)                                               #
        filt2 = cv2.imread(noseFilter[filtIdx2], cv2.IMREAD_UNCHANGED)                                                   #
        filt3 = cv2.imread(mouthFilter[filtIdx3], cv2.IMREAD_UNCHANGED)          #########################################  
        
        # centernter of face                                                     #加入兩眼中央之錨點位置方便定位------------#
        center_x = int(round((points[0][0]+points[1][0])/2))                                                             #
        center_y = int(round((points[0][1]+points[1][1])/2))                     #########################################
        
        x_start = 0
        x_stop  = 0
        y_start = 0
        y_stop  = 0
        
        # for mouth                                                                ####更換嘴巴####
        filt_width = int(math.floor((points[3][0]-points[5][0])*0.9/2))
        filt_height = int(math.floor((points[14][1]-points[10][1])*0.7/2))
        x_start = int(points[14][0]-filt_width)
        x_stop  = int(points[14][0]+filt_width)
        y_start = int(points[11][1]-filt_height)
        y_stop  = int(points[11][1]+filt_height)
        filt_resized = cv2.resize(filt3, (x_stop-x_start, y_stop-y_start))
        transparent_region = filt_resized[:,:,:3] != 0
        
        if (face_resized_color[y_start:y_stop, x_start:x_stop,:].shape[0]== \
            filt_resized[:, :, :3].shape[0]):
            face_resized_color[y_start:y_stop, x_start:x_stop,:][transparent_region]= \
            filt_resized[:,:,:3][transparent_region]
        
        else : 
            print ("x: {}:{}, y: {}:{}, filt_resized : {}\nface_resized_color[]: {}".format(x_start, x_stop, y_start, y_stop, filt_resized[:, :, :3].shape, face_resized_color[y_start:y_stop, x_start:x_stop,:].shape))
        
        #for nose                                                                  ####更換鼻子####
        filt_width = int(math.floor((points[0][0]-points[1][0])/2))
        filt_height = int(math.floor((points[13][1]-center_y)*0.8/2))
        x_start = center_x-filt_width
        x_stop  = center_x+filt_width
        y_start = center_y+int(0.3*filt_height)
        y_stop  = center_y+int(2.3*filt_height)
        filt_resized = cv2.resize(filt2, (x_stop-x_start, y_stop-y_start))
        transparent_region = filt_resized[:,:,:3] != 0
        if (filt_resized[:, :, :3].shape[0]==face_resized_color[y_start:y_stop, x_start:x_stop,:].shape[0]):
            face_resized_color[y_start:y_stop, x_start:x_stop,:][transparent_region]= filt_resized[:,:,:3][transparent_region]
        else : print ("filt_resized : {}\nface_resized_color[]: {}".format(filt_resized[:, :, :3].shape, face_resized_color[y_start:y_stop, x_start:x_stop,:].shape))
            
        # for sunglasses                                                           ####更換眼鏡或眼睛####
        filt_width = int(math.floor((points[7][0]-points[9][0])*1.1/2))
        filt_height = int(math.floor((points[10][1]-points[8][1])*0.9/2))
        x_start = center_x-filt_width
        x_stop  = center_x+filt_width
        y_start = center_y-filt_height
        y_stop  = center_y+filt_height
        filt_resized = cv2.resize(filt1, (x_stop-x_start, y_stop-y_start))
        transparent_region = filt_resized[:,:,:3] != 0
        
        if (filt_resized[:, :, :3].shape[0]==face_resized_color[y_start:y_stop, x_start:x_stop,:].shape[0]):
            face_resized_color[y_start:y_stop, x_start:x_stop,:][transparent_region]= filt_resized[:,:,:3][transparent_region]
        else : print ("filt_resized : {}\nface_resized_color[]: {}".format(filt_resized[:, :, :3].shape, face_resized_color[y_start:y_stop, x_start:x_stop,:].shape))

            
        # Resize the face_resized_color image back to its original shape
        frame[y:y+h, x:x+w] = cv2.resize(face_resized_color, original_shape, interpolation = cv2.INTER_CUBIC)

        # Add KEYPOINTS to the frame2
        for keypoint in points:
            cv2.circle(face_resized_color2, keypoint, 1, (0,255,0), 1)

        frame2[y:y+h, x:x+w] = cv2.resize(face_resized_color2, original_shape, interpolation = cv2.INTER_CUBIC)

        # Show the frame and the frame2
        cv2.imshow("Selfie Filters", frame)
        cv2.imshow("Facial Keypoints", frame2)

    # If the 'q' key is pressed, stop the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()


Using TensorFlow backend.


load model complete
load cascate complete


<h2 style="background-color:LightGrey; padding:20px; margin:0">● 執行流程與結果</h2>
<h3 style="background-color:LightGrey; padding:20px; margin:0">執行my_CNN_model.py(2.)以及shader.py(4.)之後，串流影像即開始運作。<br><br>可看見兩視窗，右方視窗為經由model預測之標記點位置，左方視窗則為濾鏡效果(如下圖所示)。</h3>

<img src="1.png" style="width:45%; float:left">
<img src="2.png" style="width:45%; float:right">
<br>
<img src="3.png" style="width:45%; float:left">
<img src="4.png" style="width:45%; float:right">
<br>
<img src="5.png" style="width:45%; float:left">
<img src="6.png" style="width:45%; float:right">